In [1]:
!pip install pythreejs
!pip install notebook
!pip install jupyter_contrib_nbextensions
!jupyter contrib nbextension install --user


  Using cached ipywidgets-8.1.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached widgetsnbextension-4.0.13-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl.metadata (4.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 10.4 MB/s eta 0:00:00a 0:00:01
Using cached ipywidgets-8.1.5-py3-none-any.whl (139 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 7.2 MB/s eta 0:00:00a 0:00:01
Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl (214 kB)
Using cached widgetsnbextension-4.0.13-py3-none-any.whl (2.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 9.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 11.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 11.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 10.0 MB/s eta 0:00:00
  

In [95]:
from pythreejs import *
from IPython.display import display

# Create a 3D sphere
sphere = Mesh(
    geometry=SphereGeometry(radius=1),
    material=MeshStandardMaterial(color='red')
)

# Rendering
width, height = 1600, 900

light = PointLight(position=[10, 10, 10], intensity=1.5)
scene = Scene(children=[sphere, light, AmbientLight(intensity=0.5)])


camera = PerspectiveCamera(position=[3, 3, 3], fov=50, aspect=width/height)
controller = OrbitControls(controlling=camera)


renderer = Renderer(camera=camera, scene=scene, controls=[controller], width=width, height=height)
display(renderer)


Renderer(camera=PerspectiveCamera(aspect=1.7777777777777777, position=(3.0, 3.0, 3.0), projectionMatrix=(1.0, …

In [99]:
from pythreejs import * 
from IPython.display import display
import numpy as np


def generate_subdivided_circle(radius=1.0, segments=16):
    """Generate vertices and triangular faces for a subdivided 2D circle."""
    vertices = []
    faces = []
    
    # Add the center vertex
    vertices.append([0.0, 0.0, 0.0])  # Center at (0, 0)
    
    # Generate boundary vertices
    for i in range(segments):
        theta = 2 * np.pi * i / segments
        x = radius * np.cos(theta)
        y = radius * np.sin(theta)
        vertices.append([x, y, 0.0])
    
    # Create faces (triangles)
    for i in range(segments):
        p1 = 0  # Center vertex
        p2 = i + 1
        p3 = (i + 1) % segments + 1  # Wrap around to the first boundary vertex
        faces.append([p1, p2, p3])
    
    return np.array(vertices, dtype=np.float32), np.array(faces, dtype=np.uint32)

# Generate sphere vertices and faces
vertices, faces = generate_subdivided_circle(radius=1.0, segments=32)
vertices[:, 1] += 3
# Create BufferGeometry using the vertices and faces
geometry = BufferGeometry(
    attributes={
        'position': BufferAttribute(array=vertices, normalized=False),
    },
    index=BufferAttribute(array=faces.ravel(), normalized=False)
)

# Define material and mesh
material = MeshStandardMaterial(color='black', wireframe=True)
mesh = Mesh(geometry=geometry, material=material)



# Draw Ground
ground_vertices = np.array([
    [-10.0, 0.0, 0.0],  # Start point
    [10.0, 0.0, 0.0]    # End point
], dtype=np.float32)
ground_geometry = BufferGeometry(
    attributes={
        'position': BufferAttribute(ground_vertices, normalized=False)
    }
)
ground_material = LineBasicMaterial(color='red', linewidth=2)
line = Line(geometry=ground_geometry, material=ground_material)



# Rendering setup
width, height = 800, 600  # Smaller dimensions for 2D
aspect = width / height
scene = Scene(children=[mesh, line, AmbientLight(intensity=0.8)])
camera = OrthographicCamera(left=-5*aspect, right=5*aspect, top=5, bottom=-5, near=0.001, far=100, position=[0, 0, 10])
controller = OrbitControls(controlling=camera, enableRotate=False)
renderer = Renderer(camera=camera, scene=scene, controls=[controller], width=width, height=height)
renderer.background = '#FFFFFF'  # Optional: Set a white background for better 2D visibility

display(renderer)

Renderer(background='#FFFFFF', camera=OrthographicCamera(bottom=-5.0, far=100.0, left=-6.666666666666666, near…

In [100]:
import numpy as np
import time

# Simulation parameters
time_step = 1/30
num_iterations = 100

max_iterations = 100
gravity = np.array([0, -9.81, 0])

# damping = 0.99

# State
positions = vertices.copy()
velocities = np.zeros_like(positions)

# Mass and inverse mass
mass = 1.0
inv_mass = 1.0 / mass

# Constraints (for example, distance constraints between vertices)
def generate_constraints(vertices, faces):
    # Step 1: Extract unique edges from faces
    edges = set()
    for face in faces:
        # Add each edge of the triangle, sorted to avoid duplicates
        edges.add(tuple(sorted((face[0], face[1]))))
        edges.add(tuple(sorted((face[1], face[2]))))
        edges.add(tuple(sorted((face[2], face[0]))))
    
    # Step 2: Generate constraints
    constraints = []
    for edge in edges:
        v1, v2 = edge
        rest_length = np.linalg.norm(vertices[v1] - vertices[v2])
        constraints.append((v1, v2, rest_length))
    
    return constraints

constraints = generate_constraints(vertices, faces)

def apply_external_force(velocities, gravity, time_step):
    v = velocities + inv_mass* gravity * time_step
    p = positions + v * time_step
    return p
    

def damp_velocities(velocities, damping):
    velocities *= damping

def solve_constraints(positions, constraints, inv_mass, num_iterations):
    for _ in range(num_iterations):
        for (i, j, rest_length) in constraints:
            p1, p2 = positions[i], positions[j]
            delta = p2 - p1
            delta_length = np.linalg.norm(delta)
            if delta_length > 0:
                correction = (delta_length - rest_length) * delta / delta_length
                positions[i] += correction * inv_mass / 2
                positions[j] -= correction * inv_mass / 2
    return positions

def integrate(positions, velocities, time_step):
    positions += velocities * time_step

# Simulation loop
for step in range(1000000):  # Run for 100 steps
    positions = apply_external_force(velocities, gravity, time_step)
    positions = solve_constraints(positions, constraints, inv_mass, num_iterations)
    
    collide = np.where(positions[:, 1] < 0)
    positions[collide, 1] = 0
    # Update velocities based on new positions
    velocities = (positions - vertices) / time_step
    vertices = positions.copy()
    
    # Dynamic friction
    velocities[collide, 0] = velocities[collide, 0] * np.maximum(1 - 0.2*time_step*abs(velocities[collide, 0]), 0)
    # Static friction
    condition = np.abs(velocities[collide, 0]) < 0.9 * np.abs(velocities[collide, 1] + 1e-6)
    velocities[collide, 0] = np.where(condition, 0, velocities[collide, 0])
    
    # Render the updated positions (this part depends on your rendering setup)
    # For example, you can update the BufferGeometry with new positions
    geometry.attributes['position'].array = positions
    geometry.attributes['position'].needs_update = True
    time.sleep(0.01)

/home/matthew/Documents/ch_kim/xpbd/.venv/lib/python3.9/site-packages/pythreejs/traits.py:257: UserWarning: 64-bit data types not supported for WebGL data, casting to 32-bit.
  warnings.warn('64-bit data types not supported for WebGL '


KeyboardInterrupt: 